In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("kindle_reviews.csv")

In [4]:
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [6]:
df = df[['reviewText','overall']]

In [72]:
df = df.head(100000)

In [73]:
df.shape

(100000, 3)

In [74]:
df.loc[df['overall']<=3, 'Sentiment']=0

C:\Users\HW798MH\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [75]:
df.loc[df['overall']>3, 'Sentiment']=1

In [76]:
df.head()

,reviewText,overall,Sentiment
0,enjoy vintage book movie enjoyed reading book ...,5,1
1,book reissue old one author born era say nero ...,4,1
2,fairly interesting read old style terminologyi...,4,1
3,id never read amy brewster mystery one really ...,5,1
4,like period piece clothing lingo enjoy mystery...,4,1


In [77]:
df.isna(). sum()

reviewText    0
overall       0
Sentiment     0
dtype: int64

In [78]:
df = df.dropna()

In [79]:
df.isna().sum()

reviewText    0
overall       0
Sentiment     0
dtype: int64

In [80]:
import nltk 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HW798MH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HW798MH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HW798MH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HW798MH\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [81]:
df['reviewText'] = df['reviewText'].str.lower()

In [82]:
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-zA-Z ]', '',x))

In [83]:
url_pattern = r'https?://\S+|www\.\S+'
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(url_pattern, '', x))

In [84]:
html_pattern = r'<.*?>'
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(html_pattern, '', x))

In [85]:
lemmatizer = WordNetLemmatizer()
def process_text(review):
    words = word_tokenize(review)
    words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]
    review = " ".join(words)
    return review

df['reviewText'] = df['reviewText'].apply(process_text)

In [86]:
df.shape

(100000, 3)

In [112]:
X = df['reviewText']
y = df['Sentiment'].astype(int)

In [113]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=123)

In [114]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,ngram_range=(1,2))
X_train_bow = cv.fit_transform(X_train).toarray()
X_test_bow = cv.transform(X_test).toarray()

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=10000,ngram_range=(1,2))
X_train_tfidf = tv.fit_transform(X_train).toarray()
X_test_tf_idf = tv.transform(X_test).toarray()

In [116]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [117]:
np.unique(y_train)

array([0, 1])

In [118]:
pd.Series(y_train).isna().sum()

0

In [119]:
y_train = y_train.values
y_test = y_test.values

In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train_bow,y_train)
y_pred = model.predict(X_test_bow)
accuracy_score(y_pred,y_test)

0.83345

In [122]:
model = RandomForestClassifier()
model.fit(X_train_tfidf,y_train)

ValueError: Found input variables with inconsistent numbers of samples: [80000, 20000]

In [124]:
y_pred = model.predict(X_test_tf_idf)
accuracy_score(y_pred,y_test)

0.8322

In [125]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [126]:
wv = KeyedVectors.load("word2vec-google-news-300.model")

In [127]:
model = gensim.models.Word2Vec(X)

In [128]:
def average_word2vec(text, model):
    # Tokenize the text into words
    words = text.split()
    
    # Initialize a list to hold the word vectors
    vectors = []
    
    # Retrieve the vector for each word and add it to the list
    for word in words:
        if word in model:
            vectors.append(model[word])
    
    # If no valid vectors were found, return None
    if not vectors:
        return None
    
    # Compute the average of the vectors
    avg_vector = np.mean(vectors, axis=0)
    
    return avg_vector

In [150]:
df = df.reset_index()

In [152]:
from tqdm import tqdm
import pandas as pd
import numpy as np

X = []
valid_labels = []

# Ensure y is reset to default integer index
y = y.reset_index(drop=True)

for i in tqdm(range(len(df))):
    avg_vector = average_word2vec(df['reviewText'][i], wv)
    if avg_vector is not None:
        X.append(avg_vector)
        valid_labels.append(y.iloc[i])

# Convert X to a pandas DataFrame
X_df = pd.DataFrame(X)
y_series = pd.Series(valid_labels)  # Ensure y matches the length of X_df

# Check shapes to confirm they are as expected
print(f"X_df shape: {X_df.shape}")  # Should be (n_samples, n_features)
print(f"y_series shape: {y_series.shape}")


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:32<00:00, 3046.53it/s]


X_df shape: (100000, 300)
y_series shape: (100000,)


In [153]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_series, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
score